In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from stop_words import get_stop_words
#from sklearn import tree
import os
from datetime import datetime
import pickle

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/MLProject_KenyaFinancial
%cd G:\My Drive/MLProject_KenyaFinancial
DATA_PATH = "G:\My Drive\MLProject_KenyaFinancial\clean_data\official.csv" #replace with your data path
os.getcwd()

G:\My Drive\MLProject_KenyaFinancial


'G:\\My Drive\\MLProject_KenyaFinancial'

In [ ]:
data = pd.read_csv(DATA_PATH)

In [ ]:
data.shape

(722068, 165)

In [ ]:
dataX = data.drop(['trx_mode_desc'], axis = 1)

In [ ]:
X = dataX.loc[:,:] 

In [ ]:
X.columns

Index(['hh_ids', 'account_ids', 'm_ids_owner', 'first_trx_date_acc',
       'last_trx_date_acc', 'trx_id', 'm_ids_trx', 'trx_date', 'trx_month',
       'trx_year',
       ...
       'dem_i_hhead', 'dem_i_marstat', 'dem_i_tribe', 'dem_i_children_tot',
       'dem_i_children_inhh', 'edu_i_enrolled', 'edu_i_attain',
       'edu_i_attain_yrs', 'trx_outlet1', 'trx_purpose1'],
      dtype='object', length=164)

In [ ]:
Y = data.trx_mode_desc

In [ ]:
for column in X.columns:
  if X[column].dtype == "object":
    X[column] = X[column].astype("|S")

In [ ]:
dropCols = []
for column in X.columns:
  if X[column].nunique() > 150 or X[column].dtype == 'float64':
    dropCols.append(column)
X = X.drop(dropCols, axis=1)

In [ ]:
dropCols

In [ ]:
#Skip and load X with pickle in later cell

from sklearn.preprocessing import OneHotEncoder

ohc = OneHotEncoder(handle_unknown='ignore', drop='if_binary')

X = ohc.fit_transform(X)

In [ ]:
X.shape

(722068, 362)

In [ ]:
with open("Xpkl_new1", 'wb') as ofile:
  pickle.dump(X, ofile)


In [ ]:
####################################### LOADING X
with open("Xpkl", 'rb') as ifile:
  X = pickle.load(ifile)

In [ ]:
############################### Program is crashing here itself due to lack of ram
xdf = pd.DataFrame(X.toarray())
#xdf.to_csv("X.csv")

In [ ]:
xdf.columns = ohc.get_feature_names_out()

In [ ]:
############################## Adding numeric columns back to X
for col in dropCols:
  xdf[col] = dataX[col]

C:\Users\krish\AppData\Local\Temp/ipykernel_15304/2936279118.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  xdf[col] = dataX[col]


In [ ]:
DC = ["first_trx_date_acc", "last_trx_date_acc", "trx_date"]
first_trx_date_acc_f = []
last_trx_date_acc_f = []
trx_date_f = []

i = 0
for dc in DC:
    
    L = []
        
    for d in xdf[dc]:
        try:
            dat = datetime.strptime(d.decode('UTF-8'), "%d%b%Y")
            f = dat.timestamp()
            L.append(f)
        except:
            L.append(np.nan)
    
    if i == 0:
        first_trx_date_acc_f = L
    if i == 1:
        last_trx_date_acc_f = L
    if i == 2:
        trx_date_f = L
        
    i += 1

In [ ]:
xdf['first_trx_date_acc_f'] = first_trx_date_acc_f
xdf["last_trx_date_acc_f"] = last_trx_date_acc_f
xdf["trx_date_f"] = trx_date_f

C:\Users\krish\AppData\Local\Temp/ipykernel_15304/795257833.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  xdf['first_trx_date_acc_f'] = first_trx_date_acc_f
C:\Users\krish\AppData\Local\Temp/ipykernel_15304/795257833.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  xdf["last_trx_date_acc_f"] = last_trx_date_acc_f
C:\Users\krish\AppData\Local\Temp/ipykernel_15304/795257833.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [ ]:
Xd = xdf.drop(DC, axis = 1)

In [ ]:
dropCol2 = []
for col in Xd.columns:
    if Xd[col].dtype != "float64":
        dropCol2.append(col)
        print(col)

hh_ids
account_ids
m_ids_owner
m_ids_trx


In [ ]:
Xd = Xd.drop(dropCol2, axis=1)

In [ ]:
#imputer
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=5) # perhaps reduce n_neighbours?

In [ ]:
# Taking too long as of now. May run later
# Have used simple imputing based on means for now
Ximp = imputer.fit_transform(Xd)

In [ ]:
# sklearn imputer is reducing number of columns for some reason. 
# Perhaps it is removing columns that are completely Nan, or if it has values it can't decode
# Anyway, I am using the pandas way of doing it instead

Ximp = Xd.fillna(Xd.mean())

In [ ]:
os.makedirs('clean_data', exist_ok=True)  # make dataset available as csv for machine learning preprocessing
compression_opts = dict(method='zip',
                        archive_name='XoheImp.csv')
Ximp.to_csv('clean_data/XoheImp.zip', index=False,
          compression=compression_opts) 

In [ ]:
with open("depVar", 'wb') as ofile:
    pickle.dump(Y, ofile)

In [ ]:
os.makedirs('clean_data', exist_ok=True)  # make dataset available as csv for machine learning preprocessing
compression_opts = dict(method='zip',
                        archive_name='Y.csv')
Y.to_csv('clean_data/Y.zip', index=False,
          compression=compression_opts) 